In [ ]:
!pip install torchvision


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from PIL import Image
import matplotlib.pyplot as plt

# Load the VGG19 model
vgg = models.vgg19(pretrained=True).features

# Freeze parameters
for param in vgg.parameters():
    param.requires_grad_(False)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg.to(device)

# Define image loading and preprocessing function
def load_image(img_path, max_size=400, shape=None):
    image = Image.open(img_path).convert('RGB')

    # Resize image to specified max_size
    if max_size:
        size = max(max_size, max(image.size))
        if shape:
            size = shape
        in_transform = transforms.Compose([
            transforms.Resize(size),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), 
                                 (0.229, 0.224, 0.225))])
        image = in_transform(image)[:3, :, :].unsqueeze(0)

    return image

# Load images
content = load_image('con.jpg').to(device)
style = load_image('style.jpg', shape=content.shape[-2:]).to(device)

# Display images
def im_convert(tensor):
    image = tensor.to("cpu").clone().detach()
    image = image.numpy().squeeze()
    image = image.transpose(1, 2, 0)
    image = image * (0.229, 0.224, 0.225) + (0.485, 0.456, 0.406)
    image = image.clip(0, 1)
    return image

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(im_convert(content))
plt.title("Content Image")
plt.subplot(1, 2, 2)
plt.imshow(im_convert(style))
plt.title("Style Image")
plt.show()

# Define content and style layers from VGG19
def get_features(image, model, layers=None):
    if layers is None:
        layers = {'0': 'conv1_1', '5': 'conv2_1', 
                  '10': 'conv3_1', '19': 'conv4_1', 
                  '21': 'conv4_2', '28': 'conv5_1'}
    features = {}
    x = image
    for name, layer in model._modules.items():
        x = layer(x)
        if name in layers:
            features[layers[name]] = x
    return features

# Calculate the Gram Matrix for style features
def gram_matrix(tensor):
    _, d, h, w = tensor.size()
    tensor = tensor.view(d, h * w)
    gram = torch.mm(tensor, tensor.t())
    return gram

# Get content and style features
content_features = get_features(content, vgg)
style_features = get_features(style, vgg)

# Calculate Gram matrices for style features
style_grams = {layer: gram_matrix(style_features[layer]) for layer in style_features}

# Create a target image to optimize
target = content.clone().requires_grad_(True).to(device)

# Define loss and weights
style_weights = {'conv1_1': 1.0, 'conv2_1': 0.8, 'conv3_1': 0.5, 
                 'conv4_1': 0.3, 'conv5_1': 0.1}
content_weight = 1  # alpha
style_weight = 1e6  # beta

# Define optimizer
optimizer = optim.Adam([target], lr=0.003)

# Define the total loss function
steps = 2000
for i in range(steps):
    target_features = get_features(target, vgg)
    content_loss = torch.mean((target_features['conv4_2'] - content_features['conv4_2']) ** 2)
    
    style_loss = 0
    for layer in style_weights:
        target_feature = target_features[layer]
        target_gram = gram_matrix(target_feature)
        style_gram = style_grams[layer]
        layer_style_loss = style_weights[layer] * torch.mean((target_gram - style_gram) ** 2)
        _, d, h, w = target_feature.shape
        style_loss += layer_style_loss / (d * h * w)
    
    total_loss = content_weight * content_loss + style_weight * style_loss
    
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    if i % 100 == 0:
        print(f"Step {i}/{steps}, Loss: {total_loss.item()}")

# Display the final stylized image
plt.imshow(im_convert(target))
plt.title("Stylized Image")
plt.show()
